In [244]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [245]:
data = pd.read_csv(r'D:\datasets\titanic\train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [246]:
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

Здесь было принято волевое решение пока оставить попытки извлечь пользу из признака Cabin. 

Хотя если обратить внимание на букву перед номером каюты и посмотреть карту внутреннего устройства корабля, то можно сделать вывод, что именно эта часть признака была бы наиболее информативной для нас, т.к. отсеки корабля, обозначенные буквой в номере, располагались на разных уровнях. 

Можно было бы попробовать пронумеровать их в порядке удаления от места размещения спасательных шлюпок в таком случае.

In [247]:
data['Sex'] = pd.get_dummies(data['Sex'], drop_first=True)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,S
887,1,1,0,19.0,0,0,30.0000,S
888,0,3,0,NaN,1,2,23.4500,S
889,1,1,1,26.0,0,0,30.0000,C


Признак Sex было решено закодировать путём нумерации, потому что столбец принимает всего два значения.
Если же мы бы вдург выяснили, что на Титанике появился пассажир третьего гендера, пришлось бы использовать one hot кодирование:)

In [248]:
embarked_dummies = pd.get_dummies(data['Embarked'], prefix='Embarked')
data = pd.concat([data, embarked_dummies], axis=1)
data = data.drop('Embarked', axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,0,0,1
887,1,1,0,19.0,0,0,30.0000,0,0,1
888,0,3,0,NaN,1,2,23.4500,0,0,1
889,1,1,1,26.0,0,0,30.0000,1,0,0


In [249]:
cleaned_data = data.dropna()

Признак Embarked закодирован методом one hot в основном из спортивного интереса. Т.к. он обозначает порт посадки пассажира, содержит больше двух возможных значений и исходя из предметоной области нет никакой существенной для нас связи между данными внутри этого столбца, способ нумерации здесь не подходит.

In [250]:
count, age_sum = [[0, 0] for _ in range(3)], [[0., 0.] for _ in range(3)]

for i in data.index:
    age = data['Age'][i]
    
    if not pd.isna(age):
        class_ind = data['Pclass'][i] - 1
        sex_ind = data['Sex'][i]
        
        count[class_ind][sex_ind] += 1
        age_sum[class_ind][sex_ind] += age
        
mean = [[age_sum[i][j]/count[i][j] for j in range(2)] for i in range(3)]

for i in data.index:
    if pd.isna(data['Age'][i]):
        class_ind = data['Pclass'][i] - 1
        sex_ind = data['Sex'][i]
        
        data.at[i, 'Age'] = mean[class_ind][sex_ind]

data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.00,1,0,7.2500,0,0,1
1,1,1,0,38.00,1,0,71.2833,1,0,0
2,1,3,0,26.00,0,0,7.9250,0,0,1
3,1,1,0,35.00,1,0,53.1000,0,0,1
4,0,3,1,35.00,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.00,0,0,13.0000,0,0,1
887,1,1,0,19.00,0,0,30.0000,0,0,1
888,0,3,0,21.75,1,2,23.4500,0,0,1
889,1,1,1,26.00,0,0,30.0000,1,0,0


Пропущенные значения из столбца Age восстановлены путём замены на средний возраст по полу в классе.

In [251]:
data.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
Survived,1.000000,-0.338481,-0.543351,-0.067485,-0.035322,0.081629,0.257307,0.168240,0.003650,-0.155660
Pclass,-0.338481,1.000000,0.131900,-0.407015,0.083081,0.018443,-0.549500,-0.243292,0.221009,0.081720
Sex,-0.543351,0.131900,1.000000,0.112851,-0.114631,-0.245489,-0.182333,-0.082853,-0.074115,0.125722
Age,-0.067485,-0.407015,0.112851,1.000000,-0.251313,-0.180705,0.118308,0.041504,-0.084069,0.008690
SibSp,-0.035322,0.083081,-0.114631,-0.251313,1.000000,0.414838,0.159651,-0.059528,-0.026354,0.070941
Parch,0.081629,0.018443,-0.245489,-0.180705,0.414838,1.000000,0.216225,-0.011069,-0.081228,0.063036
Fare,0.257307,-0.549500,-0.182333,0.118308,0.159651,0.216225,1.000000,0.269335,-0.117216,-0.166603
Embarked_C,0.168240,-0.243292,-0.082853,0.041504,-0.059528,-0.011069,0.269335,1.000000,-0.148258,-0.778359
Embarked_Q,0.003650,0.221009,-0.074115,-0.084069,-0.026354,-0.081228,-0.117216,-0.148258,1.000000,-0.496624
Embarked_S,-0.155660,0.081720,0.125722,0.008690,0.070941,0.063036,-0.166603,-0.778359,-0.496624,1.000000


Здесь видна небольшая корреляция между признаком Embarked_C и целевым значением Survived, на которую можно обратить внимание.

Если обратить внимание на то, что данный признак имеет также некоторую корреляцию с признаком Fare (тариф) и антикорреляцию с признаком Pclass, можно предположить, что в порту Cherbourg в корабль село большее число богатых пассажиров (относительно других портов, конечно:).

In [252]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=13)

In [253]:
x_train, y_train = train_data[train_data.columns.difference(['Survived'])], train_data[['Survived']]
x_test, y_test = test_data[test_data.columns.difference(['Survived'])], test_data[['Survived']]

In [254]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

In [255]:
d_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=6,
    min_samples_leaf=10
)

d_tree.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=10)

In [256]:
y_tree_pred = d_tree.predict(x_test)
y_tree_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0], dtype=int64)

In [257]:
print(accuracy_score(y_test, y_tree_pred))
print(classification_report(y_test, y_tree_pred))

0.8435754189944135
              precision    recall  f1-score   support

           0       0.86      0.89      0.88       110
           1       0.82      0.77      0.79        69

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



Гиперпараметры методов здесь были выбраны методом ~~кнута и пряника~~ оценки метрик, предоставляемых accuracy_score и classification_report.

In [258]:
knn = KNeighborsClassifier(
    n_neighbors=5
)

knn.fit(x_train, y_train['Survived'].ravel())

KNeighborsClassifier()

In [259]:
y_knn_pred = knn.predict(x_test)
y_knn_pred

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0], dtype=int64)

In [260]:
print(accuracy_score(y_test, y_knn_pred))
print(classification_report(y_test, y_knn_pred))

0.7430167597765364
              precision    recall  f1-score   support

           0       0.77      0.83      0.80       110
           1       0.69      0.61      0.65        69

    accuracy                           0.74       179
   macro avg       0.73      0.72      0.72       179
weighted avg       0.74      0.74      0.74       179



In [261]:
scaler = MinMaxScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [262]:
d_tree.fit(x_train_scaled, y_train)
y_tree_pred = d_tree.predict(x_test_scaled)

y_tree_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0], dtype=int64)

In [263]:
print(accuracy_score(y_test, y_tree_pred))
print(classification_report(y_test, y_tree_pred))

0.8435754189944135
              precision    recall  f1-score   support

           0       0.86      0.89      0.88       110
           1       0.82      0.77      0.79        69

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [264]:
knn.fit(x_train_scaled, y_train['Survived'].ravel())
y_knn_pred = knn.predict(x_test_scaled)

y_knn_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0], dtype=int64)

In [265]:
print(accuracy_score(y_test, y_knn_pred))
print(classification_report(y_test, y_knn_pred))

0.8156424581005587
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       110
           1       0.78      0.72      0.75        69

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.82      0.81       179



Нормализация существенно повлияла на результаты метода knn, предположительно потому что он оперирует непосредственно евклидовыми расстояниями между векторами признаков в н-мерном пространстве, а нормализация в первую очередь масштабирует признаки друг относительно друга.

В случае с деревом решений, в узле для принятия решения используется один конкретный признак и поведение остальных признаков никак не влияет на конкретный узел. Думаю, это основная причина, почему дерево инвариантно к нормализации признаков.

In [266]:
train_data, test_data = train_test_split(cleaned_data, test_size=0.2, random_state=13)

In [267]:
x_train, y_train = train_data[train_data.columns.difference(['Survived'])], train_data[['Survived']]
x_test, y_test = test_data[test_data.columns.difference(['Survived'])], test_data[['Survived']]

In [268]:
d_tree.fit(x_train, y_train)
y_tree_pred = d_tree.predict(x_test)

print(accuracy_score(y_test, y_tree_pred))
print(classification_report(y_test, y_tree_pred))

0.8041958041958042
              precision    recall  f1-score   support

           0       0.80      0.91      0.85        86
           1       0.82      0.65      0.73        57

    accuracy                           0.80       143
   macro avg       0.81      0.78      0.79       143
weighted avg       0.81      0.80      0.80       143



In [271]:
knn.fit(x_train, y_train['Survived'].ravel())
y_knn_pred = knn.predict(x_test)

print(accuracy_score(y_test, y_knn_pred))
print(classification_report(y_test, y_knn_pred))

0.7482517482517482
              precision    recall  f1-score   support

           0       0.76      0.85      0.80        86
           1       0.72      0.60      0.65        57

    accuracy                           0.75       143
   macro avg       0.74      0.72      0.73       143
weighted avg       0.75      0.75      0.74       143



In [275]:
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [276]:
d_tree.fit(x_train_scaled, y_train)
y_tree_pred = d_tree.predict(x_test_scaled)

print(accuracy_score(y_test, y_tree_pred))
print(classification_report(y_test, y_tree_pred))

0.8041958041958042
              precision    recall  f1-score   support

           0       0.80      0.91      0.85        86
           1       0.82      0.65      0.73        57

    accuracy                           0.80       143
   macro avg       0.81      0.78      0.79       143
weighted avg       0.81      0.80      0.80       143



In [277]:
knn.fit(x_train_scaled, y_train['Survived'].ravel())
y_knn_pred = knn.predict(x_test_scaled)

print(accuracy_score(y_test, y_knn_pred))
print(classification_report(y_test, y_knn_pred))

0.8181818181818182
              precision    recall  f1-score   support

           0       0.83      0.88      0.85        86
           1       0.80      0.72      0.76        57

    accuracy                           0.82       143
   macro avg       0.82      0.80      0.81       143
weighted avg       0.82      0.82      0.82       143



На очищенных данных результаты обучения обоих алгоритмов хуже, т.к. мы теряем часть данных.